In [0]:
%fs ls abfss://datalake@datalakegen2storagelele.dfs.core.windows.net

path,name,size,modificationTime
abfss://datalake@datalakegen2storagelele.dfs.core.windows.net/delta/,delta/,0,1697997388000
abfss://datalake@datalakegen2storagelele.dfs.core.windows.net/example/,example/,0,1697901489000


In [0]:
%fs ls /mnt/datalake/example/games

path,name,size,modificationTime
dbfs:/mnt/datalake/example/games/games.csv,games.csv,4835545,1698006177000
dbfs:/mnt/datalake/example/games/games_metadata.json,games_metadata.json,17836350,1698006188000
dbfs:/mnt/datalake/example/games/recommendations.csv,recommendations.csv,1858800401,1698006438000
dbfs:/mnt/datalake/example/games/users.csv,users.csv,183760643,1698006222000


## Annual Game Release Count


In [0]:
%scala
val df = spark.read.format("csv")
.option("header", "true")
.option("delimiter", ",")
.load("dbfs:/mnt/datalake/example/games/games.csv")
df.show()

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
app_id| title|date_release| win| mac|linux| rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
 13500|Prince of Persia:...| 2008-11-21|true|false|false| Very Positive| 84| 2199| 9.99| 9.99| 0.0| true|
 22364|BRINK: Agents of ...| 2011-08-03|true|false|false| Positive| 85| 21| 2.99| 2.99| 0.0| true|
113020|Monaco: What's Yo...| 2013-04-24|true| true| true| Very Positive| 92| 3722| 14.99| 14.99| 0.0| true|
226560| Escape Dead Island| 2014-11-18|true|false|false| Mixed| 61| 873| 14.99| 14.99| 0.0| true|
249050|Dungeon of the EN...| 2014-10-27|true| true|false| Very Positive| 88| 8784| 11.99| 11.99| 0.0| true|
250180| METAL SLUG 3| 2015-09-14|true|false|false| Very Positive| 90| 5579| 7.99| 7.99| 0.0| true|
253980| Enclave| 2013-10-04|true| true| true|Mostly Positive| 75| 1608| 4.99| 4.99| 0.0| true|
271850|Men of War: Assau...| 2014-05-16|true|false|false| Mixed| 61| 199| 6.99| 6.99| 0.0| true|
282900|Hyperdimension Ne...| 2015-01-29|true|false|false| Very Positive| 94| 9686| 14.99| 14.99| 0.0| true|
 19810|The Sum of All Fears| 2008-10-10|true|false|false|Mostly Positive| 75| 33| 9.99| 9.99| 0.0| true|
 15270| Cold Fear™| 2008-05-13|true|false|false| Very Positive| 85| 800| 9.99| 9.99| 0.0| true|
 21130|LEGO® Harry Potte...| 2010-06-25|true|false|false| Very Positive| 85| 5169| 19.99| 19.99| 0.0| true|
 22130|Hearts of Iron 2 ...| 2009-01-23|true|false|false| Very Positive| 85| 462| 14.99| 14.99| 0.0| true|
 29180| Osmos| 2009-08-18|true| true| true| Very Positive| 88| 532| 9.99| 9.99| 0.0| true|
 32750| Comanche 4| 2009-06-18|true|false|false| Very Positive| 90| 222| 9.99| 9.99| 0.0| true|
241620| Inquisitor| 2013-08-01|true|false|false|Mostly Positive| 70| 390| 9.99| 9.99| 0.0| true|
244910| Homesick| 2015-05-28|true|false|false|Mostly Positive| 77| 1139| 14.99| 14.99| 0.0| true|
245950|Borderlands 2: He...| 2014-02-11|true| true| true| Very Positive| 84| 294| 0.89| 2.99| 70.0| true|
250460| Bridge Constructor| 2013-10-16|true| true| true|Mostly Positive| 77| 716| 2.39| 19.99| 88.0| true|
278890| Angvik| 2014-02-24|true| true|false| Very Positive| 88| 1986| 2.99| 2.99| 0.0| true|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
only showing top 20 rows

df: org.apache.spark.sql.DataFrame = [app_id: string, title: string ... 11 more fields]

In [0]:
%scala
df.printSchema()

root
-- app_id: string (nullable = true)
-- title: string (nullable = true)
-- date_release: string (nullable = true)
-- win: string (nullable = true)
-- mac: string (nullable = true)
-- linux: string (nullable = true)
-- rating: string (nullable = true)
-- positive_ratio: string (nullable = true)
-- user_reviews: string (nullable = true)
-- price_final: string (nullable = true)
-- price_original: string (nullable = true)
-- discount: string (nullable = true)
-- steam_deck: string (nullable = true)

In [0]:
%scala
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._

In [0]:
%scala
val annual_release_count = df.filter(col("app_id").isNotNull and (col("app_id") =!= ""))
.groupBy(year(to_date(col("date_release"), "yyyy-MM-dd")).as("year"))
.agg(count("*").alias("count"))
.sort(col("count").desc)

annual_release_count.show()

+----+-----+
year|count|
+----+-----+
2022| 7271|
2021| 6776|
2020| 6138|
2018| 5462|
2019| 5057|
2017| 4990|
2016| 4210|
2023| 3103|
2015| 2963|
2014| 1921|
2013| 822|
2012| 565|
2011| 376|
2009| 322|
2010| 284|
2008| 146|
2007| 82|
2006| 56|
2004| 4|
2005| 3|
+----+-----+
only showing top 20 rows

annual_release_count: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [year: int, count: bigint]

In [0]:
%scala
annual_release_count.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/games/annual_release_count")

## Operating System Breakdown for Game Distribution

In [0]:
%scala
df.printSchema()

root
-- app_id: string (nullable = true)
-- title: string (nullable = true)
-- date_release: string (nullable = true)
-- win: string (nullable = true)
-- mac: string (nullable = true)
-- linux: string (nullable = true)
-- rating: string (nullable = true)
-- positive_ratio: string (nullable = true)
-- user_reviews: string (nullable = true)
-- price_final: string (nullable = true)
-- price_original: string (nullable = true)
-- discount: string (nullable = true)
-- steam_deck: string (nullable = true)

In [0]:
%scala
val operating_distribution_count = df.agg(
  sum(when($"win"=== "true", 1).otherwise(0)).alias("win_count"),
  sum(when($"mac" === "true", 1).otherwise(0)).alias("mac_count"),
  sum(when($"linux" === "true", 1).otherwise(0)).alias("linux_count")

)
operating_distribution_count.show()

+---------+---------+-----------+
win_count|mac_count|linux_count|
+---------+---------+-----------+
 49769| 12964| 9010|
+---------+---------+-----------+

operating_distribution_count: org.apache.spark.sql.DataFrame = [win_count: bigint, mac_count: bigint ... 1 more field]

In [0]:
%scala
operating_distribution_count.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/games/operating_distribution_count")

## Correlation Between Game Playtime and User Recommendations

In [0]:
%scala
val df_recommendation = spark.read.format("csv")
.option("header", "true")
.option("delimiter", ",")
.load("dbfs:/mnt/datalake/example/games/recommendations.csv")

df_recommendation: org.apache.spark.sql.DataFrame = [app_id: string, helpful: string ... 6 more fields]

In [0]:
%scala
df_recommendation.show()

+-------+-------+-----+----------+--------------+-----+-------+---------+
 app_id|helpful|funny| date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+----------+--------------+-----+-------+---------+
 975370| 0| 0|2022-12-12| true| 36.3| 49078| 0|
 304390| 4| 0|2017-02-17| false| 11.5| 2461| 1|
1085660| 2| 0|2019-11-17| true|336.5| 240559| 2|
 703080| 0| 0|2022-09-23| true| 27.4| 245798| 3|
 526870| 0| 0|2021-01-10| true| 7.9| 22629| 4|
 306130| 0| 0|2021-10-10| true| 8.6| 43219| 5|
 238960| 0| 0|2017-11-25| true|538.8| 83923| 6|
 730| 0| 0|2021-11-30| false|157.5| 60124| 7|
 255710| 0| 0|2021-05-21| true| 18.7| 335753| 8|
 289070| 0| 0|2020-05-26| true|397.5| 430344| 9|
 431960| 0| 0|2020-10-14| true| 30.3| 189377| 10|
1086940| 0| 0|2020-10-07| true| 50.0| 81591| 11|
1938090| 0| 0|2022-11-16| true| 46.7| 152852| 12|
1286830| 2| 0|2020-07-26| true| 19.3| 107678| 13|
1172620| 0| 0|2020-11-04| true| 89.1| 116515| 14|
 306130| 0| 0|2021-05-12| true| 61.1| 71726| 15|
 635260| 0| 0|2022-01-30| true|177.0| 72831| 16|
1151340| 0| 0|2020-07-01| true| 86.3| 118655| 17|
 289070| 0| 0|2020-05-29| true|244.1| 247788| 18|
 392160| 3| 0|2018-12-26| false|320.5| 387036| 19|
+-------+-------+-----+----------+--------------+-----+-------+---------+
only showing top 20 rows

In [0]:
%scala
val correlation_recommendation_time = df_recommendation.filter(col("is_recommended").isNotNull and (col("is_recommended") =!= ""))
.groupBy("is_recommended")
.agg(round(avg(col("hours")), 2).alias("average_hours_played"))

correlation_recommendation_time.show()

+--------------+--------------------+
is_recommended|average_hours_played|
+--------------+--------------------+
 false| 83.7|
 true| 107.27|
+--------------+--------------------+

correlation_recommendation_time: org.apache.spark.sql.DataFrame = [is_recommended: string, average_hours_played: double]

In [0]:
%scala
correlation_recommendation_time.printSchema()

root
-- is_recommended: string (nullable = true)
-- average_hours_played: double (nullable = true)

In [0]:
%scala
correlation_recommendation_time.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/games/correlation_recommendation_time")

## User Feedback On Steam Games

In [0]:
%scala
df.show()

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
app_id| title|date_release| win| mac|linux| rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
 13500|Prince of Persia:...| 2008-11-21|true|false|false| Very Positive| 84| 2199| 9.99| 9.99| 0.0| true|
 22364|BRINK: Agents of ...| 2011-08-03|true|false|false| Positive| 85| 21| 2.99| 2.99| 0.0| true|
113020|Monaco: What's Yo...| 2013-04-24|true| true| true| Very Positive| 92| 3722| 14.99| 14.99| 0.0| true|
226560| Escape Dead Island| 2014-11-18|true|false|false| Mixed| 61| 873| 14.99| 14.99| 0.0| true|
249050|Dungeon of the EN...| 2014-10-27|true| true|false| Very Positive| 88| 8784| 11.99| 11.99| 0.0| true|
250180| METAL SLUG 3| 2015-09-14|true|false|false| Very Positive| 90| 5579| 7.99| 7.99| 0.0| true|
253980| Enclave| 2013-10-04|true| true| true|Mostly Positive| 75| 1608| 4.99| 4.99| 0.0| true|
271850|Men of War: Assau...| 2014-05-16|true|false|false| Mixed| 61| 199| 6.99| 6.99| 0.0| true|
282900|Hyperdimension Ne...| 2015-01-29|true|false|false| Very Positive| 94| 9686| 14.99| 14.99| 0.0| true|
 19810|The Sum of All Fears| 2008-10-10|true|false|false|Mostly Positive| 75| 33| 9.99| 9.99| 0.0| true|
 15270| Cold Fear™| 2008-05-13|true|false|false| Very Positive| 85| 800| 9.99| 9.99| 0.0| true|
 21130|LEGO® Harry Potte...| 2010-06-25|true|false|false| Very Positive| 85| 5169| 19.99| 19.99| 0.0| true|
 22130|Hearts of Iron 2 ...| 2009-01-23|true|false|false| Very Positive| 85| 462| 14.99| 14.99| 0.0| true|
 29180| Osmos| 2009-08-18|true| true| true| Very Positive| 88| 532| 9.99| 9.99| 0.0| true|
 32750| Comanche 4| 2009-06-18|true|false|false| Very Positive| 90| 222| 9.99| 9.99| 0.0| true|
241620| Inquisitor| 2013-08-01|true|false|false|Mostly Positive| 70| 390| 9.99| 9.99| 0.0| true|
244910| Homesick| 2015-05-28|true|false|false|Mostly Positive| 77| 1139| 14.99| 14.99| 0.0| true|
245950|Borderlands 2: He...| 2014-02-11|true| true| true| Very Positive| 84| 294| 0.89| 2.99| 70.0| true|
250460| Bridge Constructor| 2013-10-16|true| true| true|Mostly Positive| 77| 716| 2.39| 19.99| 88.0| true|
278890| Angvik| 2014-02-24|true| true|false| Very Positive| 88| 1986| 2.99| 2.99| 0.0| true|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
only showing top 20 rows

In [0]:
%scala
val rating_count = df.filter(col("app_id").isNotNull and (col("app_id") =!= ""))
.groupBy("rating")
.count()
.sort(col("count").desc)

rating_count: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [rating: string, count: bigint]

In [0]:
%scala
rating_count.show()

+--------------------+-----+
 rating|count|
+--------------------+-----+
 Positive|13473|
 Very Positive|13000|
 Mixed|12086|
 Mostly Positive| 8690|
 Mostly Negative| 1843|
Overwhelmingly Po...| 1096|
 Negative| 303|
 Very Negative| 59|
Overwhelmingly Ne...| 14|
+--------------------+-----+

In [0]:
%scala
rating_count.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/games/rating_count")

## Steam Game Tag Distribution

In [0]:
%scala
val df_game_tag = spark.read.format("json")
.json("dbfs:/mnt/datalake/example/games/games_metadata.json")
df_game_tag.show()

+------+--------------------+--------------------+
app_id| description| tags|
+------+--------------------+--------------------+
 13500|Enter the dark un...|[Action, Adventur...|
 22364| | [Action]|
113020|Monaco: What's Yo...|[Co-op, Stealth, ...|
226560|Escape Dead Islan...|[Zombies, Adventu...|
249050|Dungeon of the En...|[Roguelike, Strat...|
250180|“METAL SLUG 3”, t...|[Arcade, Classic,...|
253980|Experience incred...|[RPG, Action, Fan...|
271850| |[Strategy, Simula...|
282900|Packed with fast-...|[Anime, JRPG, Fem...|
 19810|Lead a team of do...| [Action, Tactical]|
 15270|In a ferocious Ar...|[Horror, Action, ...|
 21130|Build the adventu...|[LEGO, Adventure,...|
 22130|Includes Hearts o...|[Strategy, Grand ...|
 29180|Enter the ambient...|[Indie, Casual, P...|
 32750|The best-selling ...|[Action, Simulati...|
241620|Action-oriented c...|[RPG, Indie, Isom...|
244910|Explore an abando...|[Atmospheric, Puz...|
245950| |[Action, RPG, FPS...|
250460|Become an apprais...|[Simulation, Casu...|
278890|Angvik is a platf...|[Indie, Platforme...|
+------+--------------------+--------------------+
only showing top 20 rows

df_game_tag: org.apache.spark.sql.DataFrame = [app_id: bigint, description: string ... 1 more field]

In [0]:
%scala
val explod_df = df_game_tag.select($"app_id", explode($"tags").alias("tag"))
explod_df.show()

+------+-----------------+
app_id| tag|
+------+-----------------+
 13500| Action|
 13500| Adventure|
 13500| Parkour|
 13500| Third Person|
 13500| Great Soundtrack|
 13500| Singleplayer|
 13500| Platformer|
 13500| Time Travel|
 13500| Atmospheric|
 13500| Classic|
 13500| Hack and Slash|
 13500|Time Manipulation|
 13500| Gore|
 13500| Fantasy|
 13500| Story Rich|
 13500| Dark|
 13500| Open World|
 13500| Controller|
 13500| Dark Fantasy|
 13500| Puzzle|
+------+-----------------+
only showing top 20 rows

explod_df: org.apache.spark.sql.DataFrame = [app_id: bigint, tag: string]

In [0]:
%scala
val tag_count =explod_df.groupBy("tag").count().orderBy(desc("count"))
tag_count.show()

+--------------+-----+
 tag|count|
+--------------+-----+
 Indie|28166|
 Singleplayer|23070|
 Action|22342|
 Adventure|20540|
 Casual|17989|
 Simulation|12939|
 2D|11539|
 Strategy|11299|
 RPG|10373|
 Atmospheric| 8941|
 Puzzle| 7650|
 Story Rich| 7336|
 Multiplayer| 6995|
 3D| 6254|
Pixel Graphics| 6083|
 Exploration| 5918|
 First-Person| 5788|
 Cute| 5772|
 Colorful| 5770|
 Free to Play| 5731|
+--------------+-----+
only showing top 20 rows

tag_count: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [tag: string, count: bigint]

In [0]:
%scala
tag_count.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/games/tag_count")

## Top 10 Days with the Most Game Reviews on Steam

In [0]:
%scala
df_recommendation.limit(10).show()

+-------+-------+-----+----------+--------------+-----+-------+---------+
 app_id|helpful|funny| date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+----------+--------------+-----+-------+---------+
 975370| 0| 0|2022-12-12| true| 36.3| 49078| 0|
 304390| 4| 0|2017-02-17| false| 11.5| 2461| 1|
1085660| 2| 0|2019-11-17| true|336.5| 240559| 2|
 703080| 0| 0|2022-09-23| true| 27.4| 245798| 3|
 526870| 0| 0|2021-01-10| true| 7.9| 22629| 4|
 306130| 0| 0|2021-10-10| true| 8.6| 43219| 5|
 238960| 0| 0|2017-11-25| true|538.8| 83923| 6|
 730| 0| 0|2021-11-30| false|157.5| 60124| 7|
 255710| 0| 0|2021-05-21| true| 18.7| 335753| 8|
 289070| 0| 0|2020-05-26| true|397.5| 430344| 9|
+-------+-------+-----+----------+--------------+-----+-------+---------+

In [0]:
%scala
val top_10_day_reviews = df_recommendation.filter(col("app_id").isNotNull and (col("app_id") =!= ""))
.filter(col("date").isNotNull and (col("date") =!= ""))
.select("app_id", "date")
.groupBy("date")
.count()
.sort(desc("count"))
.limit(10)
top_10_day_reviews.show()

+----------+------+
 date| count|
+----------+------+
2019-06-29|193332|
2022-11-22|140930|
2021-11-24|135778|
2022-11-23|134615|
2019-06-30|130128|
2021-11-25|129705|
2020-11-25|125285|
2020-11-26|116691|
2019-07-01|101642|
2022-11-24| 94313|
+----------+------+

top_10_day_reviews: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, count: bigint]

In [0]:
%scala
top_10_day_reviews.write.mode("overwrite").format("delta").save("/mnt/datalake/delta/games/top_10_day_reviews")

## Insights into User Responses to Steam Recommendation Reviews

In [0]:
%scala
df_recommendation.limit(10).show()

+-------+-------+-----+----------+--------------+-----+-------+---------+
 app_id|helpful|funny| date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+----------+--------------+-----+-------+---------+
 975370| 0| 0|2022-12-12| true| 36.3| 49078| 0|
 304390| 4| 0|2017-02-17| false| 11.5| 2461| 1|
1085660| 2| 0|2019-11-17| true|336.5| 240559| 2|
 703080| 0| 0|2022-09-23| true| 27.4| 245798| 3|
 526870| 0| 0|2021-01-10| true| 7.9| 22629| 4|
 306130| 0| 0|2021-10-10| true| 8.6| 43219| 5|
 238960| 0| 0|2017-11-25| true|538.8| 83923| 6|
 730| 0| 0|2021-11-30| false|157.5| 60124| 7|
 255710| 0| 0|2021-05-21| true| 18.7| 335753| 8|
 289070| 0| 0|2020-05-26| true|397.5| 430344| 9|
+-------+-------+-----+----------+--------------+-----+-------+---------+

In [0]:
%scala
df_recommendation.printSchema()

root
-- app_id: string (nullable = true)
-- helpful: string (nullable = true)
-- funny: string (nullable = true)
-- date: string (nullable = true)
-- is_recommended: string (nullable = true)
-- hours: string (nullable = true)
-- user_id: string (nullable = true)
-- review_id: string (nullable = true)

In [0]:
%scala
val convert_df = df_recommendation.filter(col("app_id").isNotNull and (col("app_id") =!= ""))
.withColumn("helpful", col("helpful").cast("int"))
.withColumn("funny", col("funny").cast("int"))


convert_df: org.apache.spark.sql.DataFrame = [app_id: string, helpful: int ... 6 more fields]

In [0]:
%scala
val feedback_df = convert_df.agg(sum("helpful").alias("helpful_count"), sum("funny").alias("funny_count"))

feedback_df: org.apache.spark.sql.DataFrame = [helpful_count: bigint, funny_count: bigint]

In [0]:
%scala
feedback_df.show()

+-------------+-----------+
helpful_count|funny_count|
+-------------+-----------+
 125876064| 41676097|
+-------------+-----------+

In [0]:
%scala
feedback_df.write.mode("overwrite").format("delta").save("/mnt/datalake/delta/games/feedback_df")